In [3]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '0' # 设置 GPU 编号，如果单机单卡指定一个，单机多卡指定多个 GPU 编号
# MODEL_PATH = "THUDM/glm-4-9b-chat-hf"
MODEL_PATH = "E:/DL/ZhipuAI/glm-4-9b-chat"

device = "cuda" if torch.cuda.is_available() else "cpu"

tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH, trust_remote_code=True)

query = "你好"

inputs = tokenizer.apply_chat_template([{"role": "user", "content": query}],
                                       add_generation_prompt=True,
                                       tokenize=True,
                                       return_tensors="pt",
                                       return_dict=True
                                       )

inputs = inputs.to(device)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_PATH,
    torch_dtype=torch.bfloat16,
    low_cpu_mem_usage=True,
    trust_remote_code=True,
    device_map="auto"
).eval()

gen_kwargs = {"max_length": 2500, "do_sample": True, "top_k": 1}
with torch.no_grad():
    outputs = model.generate(**inputs, **gen_kwargs)
    outputs = outputs[:, inputs['input_ids'].shape[1]:]
    print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Using unk_token, but it is not set yet.


KeyError: '<|endoftext|>'

In [3]:
import torch
from modelscope import snapshot_download, AutoModel, AutoTokenizer
import os
model_dir = snapshot_download('ZhipuAI/glm-4-9b-chat', cache_dir='E:\DL', revision='master')


2024-12-03 22:19:27,792 - modelscope - WARNING - Using branch: master as version is unstable, use with caution


In [2]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

# 设定使用的设备为CUDA，如果CUDA不可用，将回退到CPU
device = "cuda"

# 指定预训练模型的路径
mode_name_or_path = 'E:/DL/ZhipuAI/glm-4-9b-chat'

# 从指定路径加载预训练的分词器
tokenizer = AutoTokenizer.from_pretrained(mode_name_or_path, trust_remote_code=True)


# 从指定路径加载预训练的模型
# torch_dtype指定数据类型，low_cpu_mem_usage优化CPU内存使用，trust_remote_code允许加载远程代码
model = AutoModelForCausalLM.from_pretrained(
    mode_name_or_path,
    torch_dtype=torch.bfloat16,
    low_cpu_mem_usage=True,
    trust_remote_code=True
).to(device).eval()  # 将模型设置为评估模式


Using unk_token, but it is not set yet.


KeyError: '<|endoftext|>'

In [ ]:
# 定义用户输入的查询文本
query = "请介绍一下AI大模型"

# 使用分词器的apply_chat_template方法来准备输入数据
# 这个方法会根据聊天模板将用户输入格式化为模型可接受的格式
# add_generation_prompt添加生成提示，tokenize进行分词，return_tensors指定返回PyTorch张量
# return_dict指定返回字典格式，方便后续处理
inputs = tokenizer.apply_chat_template(
    [{"role": "user", "content": query}],
    add_generation_prompt=True,
    tokenize=True,
    return_tensors="pt",
    return_dict=True
)

# 将输入数据移动到指定的设备上
inputs = inputs.to(device)


In [ ]:
# 定义生成文本时的参数
gen_kwargs = {
    "max_length": 2500,  # 设置生成文本的最大长度
    "do_sample": True,  # 是否从可能的下一个词中随机选择
    "top_k": 1  # 从概率最高的k个词中选择
}

# 使用torch.no_grad()上下文管理器来禁用梯度计算，这在推理时可以减少内存使用
with torch.no_grad():
    # 使用模型的generate方法生成文本
    outputs = model.generate(**inputs, **gen_kwargs)

    # 截取生成的文本，去除开头的提示部分
    outputs = outputs[:, inputs['input_ids'].shape[1]:]

    # 使用分词器的decode方法将生成的词ID解码回文本，并打印出来
    print(tokenizer.decode(outputs[0], skip_special_tokens=True))
